# Aries Basic Controller - Sovrin Stagingnet Issuer Agent

## This notebook walks through how to issue a credential across a newly established connection with a mobile SSI agent. 

## Before running through this notebook you should run through the following notebook - [Part 2](http://localhost:8888/notebooks/Part%202%20-%20Writing%20a%20Public%20DID%20to%20the%20Sovrin%20StagingNet.ipynb).

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


## 1. Initialise a controller for Issuer

In [72]:
%autoawait
import time
import asyncio
import pprint
import sys
from termcolor import colored,cprint
from aiohttp import ClientConnectorError, ClientResponseError
from asyncio import CancelledError

from aries_basic_controller.aries_controller import AriesAgentController
   
# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8052
ADMIN_URL = "http://external-agent:8051"


IPython autoawait is `on`, and set to use `asyncio`


In [ ]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)
    

## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [50]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


## 3a. Use a previously written schema

In [51]:
schema_id = 'MGgoJXWbeupKsaHDa7s4fW:2:testabc:0.0.1'

## 3b. (OPTIONAL) Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

**Note: You will only be able to do this once unless you change the schema_name or version. Once you have a schema written to the ledger it can be reused by multiple entities**

In [52]:
# Define you schema name - must be unique on the ledger
schema_name = "SSI PyDentity Tutorial"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["fullname", "skill", "age"]

try:
    response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
    schema_id = response["schema_id"]
    print(schema_id)
except ClientConnectorError as err:
    print(err)
except ClientResponseError as err:
    print(err)


JsL2KAjxCnov3gGxLMB7cN:2:SSI PyDentity Tutorial:0.0.1


## 4. Write a Credential Definition to the Ledger

**Note: Again this can only be done once per issuer, per schema_id.**

In [53]:
try:
    response = await agent_controller.definitions.write_cred_def(schema_id)
    cred_def_id = response["credential_definition_id"]
    print(cred_def_id)
except ClientConnectorError as err:
    print(err)
except ClientResponseError as err:
    print(err)

JsL2KAjxCnov3gGxLMB7cN:3:CL:195227:default


**Note: You should be able to see both schema and definition transactions on Sovrin Stagingnet network with [Indyscan](https://indyscan.io/home/SOVRIN_STAGINGNET)**

## 5. Populate the Attribues to Issue to Idenity Holder (User)

We will issue a credential to the identity holder consisting of the following attributes:

credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "skill", "value": "PyDentity SSI Ninja"},
    {"name": "age", "value": age}
]

The notebook will ask you to input the identity holder's full name and age which will be used to populate the schema above with the identity holders attribute information.

In [79]:
name=input("Please enter your name and surname: ")
age=input("Please enter your age: ")
credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "skill", "value": "PyDentity SSI Ninja"},
    {"name": "age", "value": age}
]
print(credential_attributes)

Please enter your name and surname:  vinnie banana
Please enter your age:  14


[{'name': 'fullname', 'value': 'vinnie banana'}, {'name': 'skill', 'value': 'PyDentity SSI Ninja'}, {'name': 'age', 'value': '14'}]


## 7. Download Identity Holder Mobile SSI Wallet
Please download a mobile SSI wallet such as [Connect.me](https://connect.me/), [Trinsic](https://trinsic.id/trinsic-wallet/) or any other Aries compatible wallet.

## 8. Create a connection invitation to connect to Alices subwallet

## 8a. Check for any existing connections

In [56]:
# Check for existing connections
try:
    connection = await agent_controller.connections.get_connections()
    print("EXISTING CONNECTIONS")
    for key, value in connection.items():
        for item in value:
            print('ConnectionID:', item['connection_id'], 'Status:',item['state'])
except ClientConnectorError as err:
    print(err)
    # raise
except ClientResponseError as err:
    print(err)

EXISTING CONNECTIONS


## 8b. Create an Invitation

In [88]:
# Create Invitation
try:
    invite = await agent_controller.connections.create_invitation()
    connection_id = invite["connection_id"]
    print("Connection ID", connection_id)
    print("Invitation Message")
    invite_message = invite['invitation']
    print(invite_message)
except ClientConnectorError as err:
    print(err)
    # raise
except ClientResponseError as err:
    print(err)

Connection ID 9f8c20bd-3c1c-47f8-8036-287ccc69200c
Invitation Message
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '40b8c5f8-8cce-4675-9b74-cf56c1012c97', 'recipientKeys': ['H1sM8yU4PFTZDYThpkAjpZJXpjhRrU3ehBhGEUrVcqgZ'], 'serviceEndpoint': 'https://0f62ba86127b.ngrok.io', 'label': 'EXTERNAL'}
Connection message {'accept': 'auto', 'created_at': '2021-03-18 12:26:47.737102Z', 'state': 'invitation', 'connection_id': '9f8c20bd-3c1c-47f8-8036-287ccc69200c', 'their_role': 'invitee', 'updated_at': '2021-03-18 12:26:47.737102Z', 'invitation_key': 'H1sM8yU4PFTZDYThpkAjpZJXpjhRrU3ehBhGEUrVcqgZ', 'rfc23_state': 'invitation-sent', 'routing_state': 'none', 'invitation_mode': 'once'} 9f8c20bd-3c1c-47f8-8036-287ccc69200c
Connection message {'accept': 'auto', 'created_at': '2021-03-18 12:26:47.737102Z', 'state': 'invitation', 'connection_id': '9f8c20bd-3c1c-47f8-8036-287ccc69200c', 'their_role': 'invitee', 'updated_at': '2021-03-18 12:26:47.737102Z', 'invitation_

### Head over to [Alice](http://localhost:8888/lab/tree/Alice/Part%203%20-%20Communicating%20with%20an%20external%20agent.ipynb) again to accept the invitation

## 8c. Check if established connection is in active state

In [89]:
import time

try:
    # print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
    print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
    while STATE != 'active':
    #     print('ConnectionId {0} is not in active state yet'.format(connection_id))
        print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
        trust_ping = await agent_controller.messaging.trust_ping(connection_id,'hello!')
    #     print('Trust ping send to ConnectionId {0} to activate connection'.format(trust_ping))
        print(colored("Trust ping send to ConnectionId {0} to activate connection".format(trust_ping), "blue", attrs=["bold"]))
        time.sleep(5)

    # print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
    print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))
except ClientResponseError as err:
    print(err)

Current state for ConnectionId 9f8c20bd-3c1c-47f8-8036-287ccc69200c is active
ConnectionId: 9f8c20bd-3c1c-47f8-8036-287ccc69200c is now active. Continue with notebook


## 9. Send Credential

This sends a credential to a identity holder (User), and automates the rest of the protocol.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [90]:
try:
    record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
    record_id = record['credential_exchange_id']
    state = record['state']
    role = record['role']
    print(f"Credential exchange {record_id}, role: {role}, state: {state}")
except CancelledError as err:
    print("Asyncio CancelledError")
except:
    raise

Credential exchange a978eb04-9bfd-45c2-81ba-0159b30c67df, role: issuer, state: offer_sent
Handle Credentials
Credential exchange a978eb04-9bfd-45c2-81ba-0159b30c67df, role: issuer, state: offer_sent
Offering: [{'name': 'fullname', 'value': 'vinnie banana'}, {'name': 'skill', 'value': 'PyDentity SSI Ninja'}, {'name': 'age', 'value': '14'}]
Handle Credentials
Credential exchange a978eb04-9bfd-45c2-81ba-0159b30c67df, role: issuer, state: offer_sent
Offering: [{'name': 'fullname', 'value': 'vinnie banana'}, {'name': 'skill', 'value': 'PyDentity SSI Ninja'}, {'name': 'age', 'value': '14'}]
Handle Credentials
Credential exchange a978eb04-9bfd-45c2-81ba-0159b30c67df, role: issuer, state: request_received
Offering: [{'name': 'fullname', 'value': 'vinnie banana'}, {'name': 'skill', 'value': 'PyDentity SSI Ninja'}, {'name': 'age', 'value': '14'}]
Handle Credentials
Credential exchange a978eb04-9bfd-45c2-81ba-0159b30c67df, role: issuer, state: request_received
Offering: [{'name': 'fullname', 'val

## 10. Accept credential in [Alice's subwallet](http://localhost:8888/lab/tree/Alice/Part%203%20-%20Communicating%20with%20an%20external%20agent.ipynb)

## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [48]:
try:
    response = await agent_controller.terminate()
    print(response)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

None
